# PyCity Schools Analysis

To make sure we're looking at all available data, we can create a larger dataframe from both CSV files and then we can begin to analyze. The first dataframe we create is a district summary, which concisely show how many schools and students there are, the total budget for the entire district, averages of all math and reading scores, and percent passing students for both categories as well as an overall.
Then, by completing a school summary, we can further break down the data to show a school's type, student body, budget per student, average scores for both reading and math, as well as percent passing those subjects. With this breakdown, we can show the top performing schools, determined by an overall passing percentage (between math and reading), as well as the lowest performing schools. We can also break down the data by subject (math and reading) to show average scores of all grades (9th to 12th).
Our final analyses depicts the type of school, the ranges of how much each school spends on its students, the ranges of the student body size, and each range's average scores for both reading and math, as well as percentages for those students passing the subjects.

With these analyses and charts drawn, one of the conclusions we can draw is that the top five performing schools (performance consisting of percentage of students who pass both reading and math) are charter schools, which typically have a smaller total number of students. This might lead someone to conclude that there might be less stress on educators who teach smaller class sizes. On the other hand, we can also see that the five lowest performing schools are district schools, with student bodies almost twice those of charter schools. It seems a highschool student attending a charter school has a higher chance of passing both subjects, even though the per capita amount actually drops along with a school's student count: schools who receive the most amount of money do not always have the highest passing rates. Another conclusion we might draw is the performance of all students (9th to 12th grade) on reading compared to math. Average reading scores across all schools and grades are consistently in the 80s, whereas math scores somewhat drop. 

In [ ]:
# Dependencies and Setup
import pandas as pd
from pathlib import Path

# File to Load
school_data_to_load = Path("Resources/schools_complete.csv")
student_data_to_load = Path("Resources/students_complete.csv")

# Read School and Student Data File and store into Pandas DataFrames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset.  
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])
school_data_complete.head()

## District Summary

In [ ]:
# Calculate the total number of unique schools
school_count = len(school_data_complete["school_name"].unique())
school_count

In [ ]:
# Calculate the total number of students
student_count = len(school_data_complete["Student ID"].unique())
student_count

In [ ]:
# Calculate the total budget
total_budget = school_data["budget"].sum()
total_budget

In [ ]:
# Calculate the average (mean) math score
average_math_score = school_data_complete["math_score"].mean()
average_math_score

In [ ]:
# Calculate the average (mean) reading score
average_reading_score = school_data_complete["reading_score"].mean()
average_reading_score

In [ ]:
# Use the following to calculate the percentage of students who passed math (math scores greather than or equal to 70)
passing_math_count = school_data_complete[(school_data_complete["math_score"] >= 70)].count()["student_name"]
passing_math_percentage = passing_math_count / float(student_count) * 100
passing_math_percentage

In [ ]:
# Calculate the percentage of students who passed reading
passing_reading_count = school_data_complete[(school_data_complete["reading_score"] >=70)].count()["student_name"]
passing_reading_percentage = passing_reading_count / float(student_count) * 100
passing_reading_percentage

In [ ]:
# Use the following to calculate the percentage of students that passed math and reading
passing_math_reading_count = school_data_complete[(school_data_complete["math_score"] >= 70) & 
                                                  (school_data_complete["reading_score"] >= 70)].count()["student_name"]
overall_passing_rate = passing_math_reading_count /  float(student_count) * 100
overall_passing_rate

In [ ]:
# Create a high-level snapshot of the district's key metrics in a DataFrame
    # Using a dictionary of lists
district_summary = pd.DataFrame({"Total Schools": [school_count], "Total Students": [student_count],
                                "Total Budget": [total_budget], "Average Math Score": [average_math_score],
                                "Average Reading Score": [average_reading_score], "% Passing Math": [passing_math_percentage],
                                "% Passing Reading": [passing_reading_percentage], "% Overall Passing": [overall_passing_rate]})


# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

## School Summary

In [ ]:
# Use the code provided to select all of the school types
school_types = school_data.groupby(["school_name"])["type"].unique()
pd.Series(school_types, name = 'type')

In [ ]:
# Calculate the total student count per school
per_school_counts = school_data_complete.groupby(["school_name"])["size"].size()
pd.Series(per_school_counts, name = 'size')

In [ ]:
# Calculate the total school budget
per_school_budget = school_data.groupby(["school_name"])["budget"].unique()
per_school_budget = per_school_budget.astype(int)
pd.Series(per_school_budget, name = 'budget')

In [ ]:
# Calculate per capita spending per school
per_school_capita = per_school_budget/per_school_counts
pd.Series(per_school_capita, name = 'Per Student Budget')

In [ ]:
# Calculate the average test scores per school
per_school_math = school_data_complete.groupby(["school_name"])["math_score"].mean()
per_school_math
per_school_reading = school_data_complete.groupby(["school_name"])["reading_score"].mean()
per_school_reading

In [ ]:
# Calculate the number of students per school with math scores of 70 or higher
students_passing_math = school_data_complete[(school_data_complete["math_score"] >= 70)]
school_students_passing_math = students_passing_math.groupby(["school_name"])["math_score"].size()
school_students_passing_math

In [ ]:
# Calculate the number of students per school with reading scores of 70 or higher
students_passing_reading = school_data_complete[(school_data_complete["reading_score"] >= 70)]
school_students_passing_reading = students_passing_reading.groupby(["school_name"])["reading_score"].size()
school_students_passing_reading

In [ ]:
# Use the provided code to calculate the number of students per school that passed both math and reading with scores of 70 or higher
students_passing_math_and_reading = school_data_complete[(school_data_complete["reading_score"] >= 70) & (school_data_complete["math_score"] >= 70)]
school_students_passing_math_and_reading = students_passing_math_and_reading.groupby(["school_name"]).size()
pd.Series(school_students_passing_math_and_reading, name = "Passed Both")

In [ ]:
# Use the provided code to calculate the passing rates
per_school_passing_math = school_students_passing_math / per_school_counts * 100
per_school_passing_reading = school_students_passing_reading / per_school_counts * 100
overall_passing_rate = school_students_passing_math_and_reading / per_school_counts * 100
pd.Series(per_school_passing_math, name = "% Passing Math")
pd.Series(per_school_passing_reading, name = "% Passing Reading")
pd.Series(overall_passing_rate, name = "% Overall Passing")

In [ ]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary_rough = pd.concat([school_types, per_school_counts, per_school_budget, per_school_capita, per_school_math, per_school_reading, per_school_passing_math, per_school_passing_reading, overall_passing_rate], axis=1)

# Rename Columns
per_school_summary = per_school_summary_rough.rename(columns={'type':'School Type', 'size':'Total Students', 'budget':'Total School Budget', 0:'Per Student Budget', 'math_score':'Average Math Score','reading_score':'Average Reading Score', 1:'% Passing Math', 2:'% Passing Reading', 3:'% Overall Passing'})

# Formatting
# per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
# per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

## Highest-Performing Schools (by % Overall Passing)

In [ ]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows.
top_schools = per_school_summary.sort_values(by=["% Overall Passing"], ascending=False)
top_schools.head(5)

## Bottom Performing Schools (By % Overall Passing)

In [ ]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
bottom_schools = per_school_summary.sort_values(by=["% Overall Passing"], ascending=True)
bottom_schools.head(5)

## Math Scores by Grade

In [ ]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the `math_score` column for each.
ninth_grade_math_scores = ninth_graders.groupby(["school_name"])
ninth_grade_math_scores_groupby = pd.Series(ninth_grade_math_scores["math_score"].mean(), name = '9th')
ninth_grade_math_scores_df = ninth_grade_math_scores_groupby.to_frame()

tenth_grade_math_scores = tenth_graders.groupby(["school_name"])
tenth_grade_math_scores_groupby = pd.Series(tenth_grade_math_scores["math_score"].mean(), name = '10th')

eleventh_grade_math_scores = eleventh_graders.groupby(["school_name"])
eleventh_grade_math_scores_groupby = pd.Series(eleventh_grade_math_scores["math_score"].mean(), name = '11th')

twelfth_grade_math_scores = twelfth_graders.groupby(["school_name"])
twelfth_grade_math_scores_groupby = pd.Series(twelfth_grade_math_scores["math_score"].mean(), name = '12th')

# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = ninth_grade_math_scores_df.merge(tenth_grade_math_scores_groupby, left_index=True, right_index=True).merge(eleventh_grade_math_scores_groupby, left_index=True, right_index=True).merge(twelfth_grade_math_scores_groupby, left_index=True, right_index=True)

# Minor data wrangling
math_scores_by_grade.index.name = None

# Display the DataFrame
math_scores_by_grade

## Reading Score by Grade 

In [ ]:
# Use the code provided to separate the data by grade
ninth_graders = school_data_complete[(school_data_complete["grade"] == "9th")]
tenth_graders = school_data_complete[(school_data_complete["grade"] == "10th")]
eleventh_graders = school_data_complete[(school_data_complete["grade"] == "11th")]
twelfth_graders = school_data_complete[(school_data_complete["grade"] == "12th")]

# Group by `school_name` and take the mean of the the `reading_score` column for each.
ninth_grade_reading_scores = ninth_graders.groupby(["school_name"])
ninth_grade_reading_scores_groupby = pd.Series(ninth_grade_reading_scores["reading_score"].mean(), name='9th')
ninth_grade_reading_scores_df = ninth_grade_reading_scores_groupby.to_frame()

tenth_grade_reading_scores = tenth_graders.groupby(["school_name"])
tenth_grade_reading_scores_groupby = pd.Series(tenth_grade_reading_scores["reading_score"].mean(), name='10th')

eleventh_grade_reading_scores = eleventh_graders.groupby(["school_name"])
eleventh_grade_reading_scores_groupby = pd.Series(eleventh_grade_reading_scores["reading_score"].mean(), name='11th')

twelfth_grade_reading_scores = twelfth_graders.groupby(["school_name"])
twelfth_grade_reading_scores_groupby = pd.Series(twelfth_grade_reading_scores["reading_score"].mean(), name='12th')

# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = ninth_grade_reading_scores_df.merge(tenth_grade_reading_scores_groupby, left_index=True, right_index=True).merge(eleventh_grade_reading_scores_groupby, left_index=True, right_index=True).merge(twelfth_grade_reading_scores_groupby, left_index=True, right_index=True)

# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

# Display the DataFrame
reading_scores_by_grade

## Scores by School Spending

In [ ]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [ ]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df = per_school_summary.copy()
school_spending_df

In [ ]:
# Use `pd.cut` to categorize spending based on the bins.
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df['Per Student Budget'], spending_bins, labels=labels)

school_spending_df

In [ ]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Math Score"].mean()
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"])["Average Reading Score"].mean()
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Math"].mean()
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Passing Reading"].mean()
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"])["% Overall Passing"].mean()

In [ ]:
# Assemble into DataFrame
spending_summary = pd.merge(spending_math_scores,spending_reading_scores,on = ["Spending Ranges (Per Student)"],how = "outer").merge(spending_passing_math,on = ["Spending Ranges (Per Student)"],how = "outer").merge(spending_passing_reading,on = ["Spending Ranges (Per Student)"],how = "outer").merge(overall_passing_spending,on = ["Spending Ranges (Per Student)"],how = "outer")

# Display results
spending_summary

## Scores by School Size

In [ ]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [ ]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.
per_school_summary["School Size"] = pd.cut(per_school_summary["Total Students"], size_bins, labels=labels)
per_school_summary.head()

In [ ]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"])["Average Math Score"].mean()
size_reading_scores = per_school_summary.groupby(["School Size"])["Average Reading Score"].mean()
size_passing_math = per_school_summary.groupby(["School Size"])["% Passing Math"].mean()
size_passing_reading = per_school_summary.groupby(["School Size"])["% Passing Reading"].mean()
size_overall_passing = per_school_summary.groupby(["School Size"])["% Overall Passing"].mean()

In [ ]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = size_summary = pd.merge(size_math_scores,size_reading_scores,on = ["School Size"],how = "outer").merge(size_passing_math,on = ["School Size"],how = "outer").merge(size_passing_reading,on = ["School Size"],how = "outer").merge(size_overall_passing,on = ["School Size"],how = "outer") 

# Display results
size_summary

## Scores by School Type

In [ ]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
average_math_score_by_type = per_school_summary.groupby(["School Type"])["Average Math Score"].mean()
average_reading_score_by_type = per_school_summary.groupby(["School Type"])["Average Reading Score"].mean()
average_percent_passing_math_by_type = per_school_summary.groupby(["School Type"])["% Passing Math"].mean()
average_percent_passing_reading_by_type = per_school_summary.groupby(["School Type"])["% Passing Reading"].mean()
average_percent_overall_passing_by_type = per_school_summary.groupby(["School Type"])["% Overall Passing"].mean()

In [ ]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary = pd.merge(average_math_score_by_type, average_reading_score_by_type,on = ["School Type"],how = "outer").merge(average_percent_passing_math_by_type,on = ["School Type"],how = "outer").merge(average_percent_passing_reading_by_type,on = ["School Type"],how = "outer").merge(average_percent_overall_passing_by_type,on = ["School Type"],how = "outer") 

# Display results
type_summary